In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

import time
import pandas as pd
import numpy as np

## PJT_03 에서 뽑은 서울데이터 가져오기

In [2]:
hos_seoul = pd.read_excel('./data/hos_seoul.xlsx')

In [3]:
hos_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영업상태명    546 non-null    object 
 1   도로명전체주소  546 non-null    object 
 2   사업장명     546 non-null    object 
 3   좌표정보(x)  526 non-null    float64
 4   좌표정보(y)  526 non-null    float64
 5   의료기관종별명  546 non-null    object 
 6   진료과목내용   546 non-null    object 
 7   진료과목내용명  546 non-null    object 
dtypes: float64(2), object(6)
memory usage: 34.2+ KB


In [4]:
hos_seoul.head()

,영업상태명,도로명전체주소,사업장명,좌표정보(x),좌표정보(y),의료기관종별명,진료과목내용,진료과목내용명
0,영업/정상,"서울특별시 중랑구 망우로 511 (망우동, 동부제일병원)",의료법인풍산의료재단동부제일병원,209677.312834,455706.658891,종합병원,101 102 104 105 106 109 110 111 116 119 122,"내과, 신경과, 외과, 정형외과, 신경외과, 마취통증의학과, 산부인과, 소아청소년과..."
1,영업/정상,서울특별시 중랑구 면목로 473 (상봉동),강북21세기병원,207506.248550,454647.912047,병원,116 109 106 105 101,"내과, 정형외과, 신경외과, 마취통증의학과, 영상의학과"
2,영업/정상,"서울특별시 중랑구 용마산로117길 49, 장스여성병원 비동 (7층 일부제외)동 (망우동)",장스여성병원,208547.560589,455271.790328,병원,110 111 101 116 119,"내과, 산부인과, 소아청소년과, 영상의학과, 진단검사의학과"
3,영업/정상,서울특별시 중랑구 사가정로49길 53 (면목동),녹색병원,207545.431398,453474.580080,종합병원,125 124 122 120 119 116 115 111 110 109 106 10...,"내과, 신경과, 정신건강의학과, 외과, 정형외과, 신경외과, 마취통증의학과, 산부인..."
4,영업/정상,"서울특별시 중랑구 망우로 474 (망우동, 조앤조병원)",조앤조병원,209177.682798,455333.044482,병원,101 104 105 106 109 114 115 116 120,"내과, 외과, 정형외과, 신경외과, 마취통증의학과, 피부과, 비뇨의학과, 영상의학과..."


## 가지고 있는 데이터로 위도경도 정보 가져오기 (카카오톡 API 활용)

- 사업장명으로 위도경도 추출하기

In [6]:
location_counts = hos_seoul['사업장명'].value_counts()

type(location_counts)

pandas.core.series.Series

In [7]:
location_counts_df = pd.DataFrame(location_counts)
type(location_counts_df)
location_counts_df.head()

# location_counts_df.to_excel('./files/3_location_counts.xlsx', index =True)

,사업장명
참튼튼병원,3
경희생생한방병원,2
서울병원,2
미올한방병원,2
서울대효병원,2


In [8]:
locations = list(location_counts_df.index)
len(locations)

534

- 사업장명으로 데이터 추출하려고 했으나 534개만 나와서 도로명주소로 재추출

In [10]:
hos_address = hos_seoul['도로명전체주소'].str.split('(').str[0]

In [11]:
hos_seoul['주소'] = hos_seoul['도로명전체주소'].str.split('(').str[0]

In [12]:
adress_01 = hos_seoul['도로명전체주소'].str.split(',').str[0]
adress_01

0          서울특별시 중랑구 망우로 511 (망우동
1         서울특별시 중랑구 면목로 473 (상봉동)
2           서울특별시 중랑구 용마산로117길 49
3      서울특별시 중랑구 사가정로49길 53 (면목동)
4          서울특별시 중랑구 망우로 474 (망우동
                  ...            
541             서울특별시 강서구 화곡로 188
542          서울특별시 강동구 고덕로 295-60
543             서울특별시 광진구 자양로 116
544            서울특별시 서초구 강남대로 309
545             서울특별시 강남구 학동로 233
Name: 도로명전체주소, Length: 546, dtype: object

In [13]:
adress_01.str.split('(').str[0]

0         서울특별시 중랑구 망우로 511 
1         서울특별시 중랑구 면목로 473 
2      서울특별시 중랑구 용마산로117길 49
3      서울특별시 중랑구 사가정로49길 53 
4         서울특별시 중랑구 망우로 474 
               ...          
541        서울특별시 강서구 화곡로 188
542     서울특별시 강동구 고덕로 295-60
543        서울특별시 광진구 자양로 116
544       서울특별시 서초구 강남대로 309
545        서울특별시 강남구 학동로 233
Name: 도로명전체주소, Length: 546, dtype: object

In [14]:
hos_seoul['뉴주소'] = adress_01.str.split('(').str[0]

In [15]:
hos_address

0                         서울특별시 중랑구 망우로 511 
1                         서울특별시 중랑구 면목로 473 
2          서울특별시 중랑구 용마산로117길 49, 장스여성병원 비동 
3                      서울특별시 중랑구 사가정로49길 53 
4                         서울특별시 중랑구 망우로 474 
                       ...                  
541         서울특별시 강서구 화곡로 188, 더와이즈황병원 3~8층 
542          서울특별시 강동구 고덕로 295-60, 원케어메디컬타운 
543    서울특별시 광진구 자양로 116, 301~307호,310~312호 
544        서울특별시 서초구 강남대로 309, 코리아비지니스센타 2층 
545     서울특별시 강남구 학동로 233, 디와이빌딩 3층 일부,4~6층 
Name: 도로명전체주소, Length: 546, dtype: object

In [16]:
hos_seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영업상태명    546 non-null    object 
 1   도로명전체주소  546 non-null    object 
 2   사업장명     546 non-null    object 
 3   좌표정보(x)  526 non-null    float64
 4   좌표정보(y)  526 non-null    float64
 5   의료기관종별명  546 non-null    object 
 6   진료과목내용   546 non-null    object 
 7   진료과목내용명  546 non-null    object 
 8   주소       546 non-null    object 
 9   뉴주소      546 non-null    object 
dtypes: float64(2), object(8)
memory usage: 42.8+ KB


### 본격 API로 위도경도 가져오기

In [17]:
import requests

In [18]:
searching = '서울특별시 중랑구 면목로 473'
url = 'http://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
url

'http://dapi.kakao.com/v2/local/search/keyword.json?query=서울특별시 중랑구 면목로 473'

In [19]:
headers = {
    "Authorization": "KakaoAK * " 
}

In [20]:
places = requests.get(url, headers = headers).json()['documents']

places[1]['place_name']
print('경도 = ',places[0]['x'])
print('위도 = ',places[0]['y'])

경도 =  127.08554886009505
위도 =  37.59403577694711


In [21]:
def find_places(searching):
    url = 'http://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {
    "Authorization": "KakaoAK b37533f330aa94672bfb49ee94dc4dbf"
    }
    places = requests.get(url, headers = headers).json()['documents']
    
    place = places[0]
    
    name = place['place_name']
    x = place['x']
    y = place['y']
    
    data = [name, x, y, searching]
    
    return(data)
  

In [22]:
data = find_places('의료법인풍산의료재단동부제일병원')
data

['동부제일병원', '127.109024676889', '37.600665739231', '의료법인풍산의료재단동부제일병원']

In [23]:
len(hos_address)

546

### 도로명 주소로 경도위도 정보가져오기

In [24]:
# ! pip install tqdm
from tqdm.notebook import tqdm

In [25]:
location = []
for i in tqdm(hos_seoul['뉴주소']):
    try:
        temp = find_places(i)
        location.append(temp)
        time.sleep(1)
    except:
        pass


  0%|          | 0/546 [00:00<?, ?it/s]

In [26]:
len(location)

545

In [27]:
hos_location = pd.DataFrame(location)
hos_location.columns = ['이름', '경도', '위도', '주소']
hos_location.head()

,이름,경도,위도,주소
0,동부제일병원,127.109024676889,37.600665739231,서울특별시 중랑구 망우로 511
1,강북21세기병원,127.08554886009505,37.59403577694711,서울특별시 중랑구 면목로 473
2,장스여성병원,127.09756336144494,37.59970990837524,서울특별시 중랑구 용마산로117길 49
3,녹색병원,127.085998786099,37.5835551077765,서울특별시 중랑구 사가정로49길 53
4,조앤조병원,127.104730044244,37.6003056281674,서울특별시 중랑구 망우로 474


In [96]:
hos_location.to_excel('./hos_location_new.xlsx', index = False)

In [3]:
hos_location_df = pd.read_excel('./hos_location_new.xlsx')
hos_location_df.head()
hos_location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   이름      545 non-null    object 
 1   경도      545 non-null    float64
 2   위도      545 non-null    float64
 3   주소      545 non-null    object 
dtypes: float64(2), object(2)
memory usage: 17.2+ KB


### 시각화를 해보자

In [4]:
import folium
import json

In [5]:
from folium.plugins import MarkerCluster

In [8]:
locations = []
names = []

for i in range(len(hos_location_df)):
    data = hos_location_df.iloc[i]
    locations.append([data['위도'], data['경도']])
    names.append(data['이름'])
    
# locations
# names

cityhall = [37.56105187605495, 126.98712420002052]
map_hos = folium.Map(location = cityhall, zoom_start = 10)


tiles = ['stamenwatercolor', 'cartodbpositron', 
         'Stamen Toner', 'stamenterrain']

for tile in tiles:
    folium.TileLayer(tile).add_to(map_hos)
    
marker_cluster = MarkerCluster(locations = locations,
                              popup = names,
                              name = 'hos',
                              overlay = True,
                              tooltip = data['이름'],
                              control = True).add_to(map_hos)

folium.LayerControl().add_to(map_hos)

map_hos

In [7]:
tiles = ['stamenwatercolor', 'cartodbpositron', 
         'openstreetmap', 'stamenterrain']

hos_map = folium.Map(location = [37.573050, 126.979189],
                    tiles = 'StamenTerrain',
                    zoom_start = 11)

for idx in hos_location_df.index:
    
    lat = hos_location_df.loc[idx, '위도']
    lng = hos_location_df.loc[idx, '경도']
    names = hos_location_df.loc[idx, '이름']
    address = '<pre>'+ hos_location_df.loc[idx, '주소'] +'</pre>'
    
    folium.CircleMarker(
    location = [lat, lng],
    popup = address,
    tooltip = names,
    radius = 4,
    weight = 1,
    fill_color = 'y',
    fill_opacity = 0.5).add_to(hos_map)
    
hos_map

In [ ]:
tiles = ['stamenwatercolor', 'cartodbpositron', 
         'Stamen Toner', 'stamenterrain']

starbucks_map = folium.Map(
    location=[37.573050, 126.979189],
    tiles='Stamen Terrain',
    zoom_start=11
)


for idx in seoul_starbucks.index:
    
    lat = seoul_starbucks.loc[idx, '위도']
    lng = seoul_starbucks.loc[idx, '경도']
    store_type = seoul_starbucks.loc[idx, '매장타입']
    names = seoul_starbucks.loc[idx, '매장명']
    address = seoul_starbucks.loc[idx, '주소']
    
    if store_type == 'general':
        fillColor = 'gray'
        
    elif store_type == 'reserve':
        fillColor = 'blue'
        
    elif store_type == 'generalDT':
        fillColor = 'red'
        
        
    folium.CircleMarker(
    location = [lat, lng],
    color = fillColor,
    tooltip = names,
    popup = '<pre>'+address+'</pre>',
    radius = 4,
    weight = 1,
    fill = True,
    fill_color = fillColor,
    fill_opacity = 0.5).add_to(starbucks_map)
    
    

starbucks_map

In [39]:
seoul = hos_seoul.copy()
seoul.head()

,영업상태명,도로명전체주소,사업장명,좌표정보(x),좌표정보(y),의료기관종별명,진료과목내용,진료과목내용명,주소,뉴주소
0,영업/정상,"서울특별시 중랑구 망우로 511 (망우동, 동부제일병원)",의료법인풍산의료재단동부제일병원,209677.312834,455706.658891,종합병원,101 102 104 105 106 109 110 111 116 119 122,"내과, 신경과, 외과, 정형외과, 신경외과, 마취통증의학과, 산부인과, 소아청소년과...",서울특별시 중랑구 망우로 511,서울특별시 중랑구 망우로 511
1,영업/정상,서울특별시 중랑구 면목로 473 (상봉동),강북21세기병원,207506.248550,454647.912047,병원,116 109 106 105 101,"내과, 정형외과, 신경외과, 마취통증의학과, 영상의학과",서울특별시 중랑구 면목로 473,서울특별시 중랑구 면목로 473
2,영업/정상,"서울특별시 중랑구 용마산로117길 49, 장스여성병원 비동 (7층 일부제외)동 (망우동)",장스여성병원,208547.560589,455271.790328,병원,110 111 101 116 119,"내과, 산부인과, 소아청소년과, 영상의학과, 진단검사의학과","서울특별시 중랑구 용마산로117길 49, 장스여성병원 비동",서울특별시 중랑구 용마산로117길 49
3,영업/정상,서울특별시 중랑구 사가정로49길 53 (면목동),녹색병원,207545.431398,453474.580080,종합병원,125 124 122 120 119 116 115 111 110 109 106 10...,"내과, 신경과, 정신건강의학과, 외과, 정형외과, 신경외과, 마취통증의학과, 산부인...",서울특별시 중랑구 사가정로49길 53,서울특별시 중랑구 사가정로49길 53
4,영업/정상,"서울특별시 중랑구 망우로 474 (망우동, 조앤조병원)",조앤조병원,209177.682798,455333.044482,병원,101 104 105 106 109 114 115 116 120,"내과, 외과, 정형외과, 신경외과, 마취통증의학과, 피부과, 비뇨의학과, 영상의학과...",서울특별시 중랑구 망우로 474,서울특별시 중랑구 망우로 474


In [40]:
seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영업상태명    546 non-null    object 
 1   도로명전체주소  546 non-null    object 
 2   사업장명     546 non-null    object 
 3   좌표정보(x)  526 non-null    float64
 4   좌표정보(y)  526 non-null    float64
 5   의료기관종별명  546 non-null    object 
 6   진료과목내용   546 non-null    object 
 7   진료과목내용명  546 non-null    object 
 8   주소       546 non-null    object 
 9   뉴주소      546 non-null    object 
dtypes: float64(2), object(8)
memory usage: 42.8+ KB


In [41]:
seoul.drop(['사업장명', '진료과목내용', '좌표정보(x)', '좌표정보(y)'], axis = 1, inplace = True)
seoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   영업상태명    546 non-null    object
 1   도로명전체주소  546 non-null    object
 2   의료기관종별명  546 non-null    object
 3   진료과목내용명  546 non-null    object
 4   주소       546 non-null    object
 5   뉴주소      546 non-null    object
dtypes: object(6)
memory usage: 25.7+ KB


In [42]:
seoul['의료기관종별명'].unique()

array(['종합병원', '병원', '요양병원(일반요양병원)', '치과병원', '한방병원', '요양병원(장애인의료재활시설)',
       '요양병원(노인병원)', '요양병원(정신병원)'], dtype=object)